## <center>Calculate Inception Score (IS) using samples generated with `Linear`, `Cosine`, and `Sigmoid` scheduler</center>

In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets, transforms, models
from torchvision.models.inception import inception_v3

import os
import pathlib
from PIL import Image 
import numpy as np
from scipy.stats import entropy

In [2]:
def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    
    N = len(imgs)
    
    assert batch_size > 0
    assert N > batch_size
    
    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor
        
    # Set up dataloader
    dataloader = DataLoader(imgs, batch_size=batch_size)
    
    # Load inception model
    inception_model = inception_v3(weights=models.Inception_V3_Weights.DEFAULT, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).detach().cpu().numpy()
    
    # Get predictions
    preds = np.zeros((N, 1000))
    
    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]
        
        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)
        
    # Now compute the mean kl-div
    split_scores = []
    
    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))
        
    return np.mean(split_scores)

In [3]:
class ImagePathDataset(Dataset):
    def __init__(self, files, transform=None):
        
        self.files = files
        self.transform = transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        path = self.files[i]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img

In [4]:
IMAGE_EXTENSIONS = {'jpg'}

In [5]:
gen_path_linear = 'GEN_Linear'
print('Total images in gen_path_linear:', len(next(os.walk(gen_path_linear))[2]))

path_linear = pathlib.Path(gen_path_linear)
files_linear = sorted([file for ext in IMAGE_EXTENSIONS for file in path_linear.glob('*.{}'.format(ext))])

dataset_linear = ImagePathDataset(files_linear, transform=transforms.ToTensor())

print("Calculating Inception Score for Linear Scheduler")
print(round(inception_score(dataset_linear, cuda=True, batch_size=50, resize=True, splits=10), 3))

Total images in gen_path_linear: 11000
Calculating Inception Score for Linear Scheduler
1.875


In [6]:
gen_path_cosine = 'GEN_Cosine'
print('Total images in gen_path_cosine:', len(next(os.walk(gen_path_cosine))[2]))

path_cosine = pathlib.Path(gen_path_cosine)
files_cosine = sorted([file for ext in IMAGE_EXTENSIONS for file in path_cosine.glob('*.{}'.format(ext))])

dataset_cosine = ImagePathDataset(files_cosine, transform=transforms.ToTensor())

print("Calculating Inception Score for Cosine Scheduler")
print(round(inception_score(dataset_cosine, cuda=True, batch_size=50, resize=True, splits=10), 3))

Total images in gen_path_cosine: 11000
Calculating Inception Score for Cosine Scheduler
1.065


In [7]:
gen_path_sigmoid = 'GEN_Sigmoid'
print('Total images in gen_path_sigmoid:', len(next(os.walk(gen_path_sigmoid))[2]))

path_sigmoid = pathlib.Path(gen_path_sigmoid)
files_sigmoid = sorted([file for ext in IMAGE_EXTENSIONS for file in path_sigmoid.glob('*.{}'.format(ext))])

dataset_sigmoid = ImagePathDataset(files_sigmoid, transform=transforms.ToTensor())

print("Calculating Inception Score for Sigmoid Scheduler")
print(round(inception_score(dataset_sigmoid, cuda=True, batch_size=50, resize=True, splits=10), 3))

Total images in gen_path_sigmoid: 11000
Calculating Inception Score for Sigmoid Scheduler
1.918
